In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [2]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_xyzConcurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [3]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [4]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [5]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [6]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [7]:
@my_decorator
def inference(thread, payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
    print(f"Prompt {thread}==> {payload_data['prompt']} \n {response.text}")
    return response, payload_data['prompt'], thread
    
        

In [8]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    index=0
    global df
    for i in range(1):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
            futures = [executor.submit(inference, srno, {**payload, 'prompt': prompt}) for srno,prompt in enumerate(prompts, start=index)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-0c8c01cfc52b48c4bc37082d18ea0360","object":"text_completion","created":520071,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 4.826624 seconds 


Prompt 6==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-c4e6a70b284c4ca2b37902c25c0c9e61","object":"text_completion","created":520071,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapsho

Prompt 7==> Problem: Write a function in python to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-02c4f02f10b541d3a5c36331788d2c70","object":"text_completion","created":520071,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef binary_search(arr, target):\n    left, right = 0, len(arr) - 1\n    while left <= right:\n        mid = (left + right) // 2\n        if arr[mid] == target:\n            return mid\n        elif arr[mid] < target:\n            left = mid + 1\n        else:\n            right = mid - 1\n    return -1\n\n# Test the function\narr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\ntarget = 5\nprint(binary_search(arr, target))  # Output: 4\n\ntarget = 11\nprint(binary_search(arr, target))  # Output: -1","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":47

Execution time: 15.902776 seconds 


Execution time: 16.163186 seconds 


Prompt 28==> Problem: Write a function in c++ to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-f89bb09331c54eb6a6a9e755ec46d487","object":"text_completion","created":520072,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple implementation of binary search in C++:\n\n```cpp\n#include <iostream>\n#include <vector>\n\nint binarySearch(std::vector<int>& arr, int target) {\n    int left = 0;\n    int right = arr.size() - 1;\n\n    while (left <= right) {\n        int mid = left + (right - left) / 2;\n\n        if (arr[mid] == target)\n            return mid;\n\n        if (arr[mid] < target)\n            left = mid + 1;\n        else\n            right = mid - 1;\n    }\n\n    return -1;\n}

Prompt 35==> Problem: Write a function in javascript to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-25d89471efa0487ba09254a787d1fe3f","object":"text_completion","created":520079,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence is a series of numbers in which each number is the sum of the two preceding ones, usually starting with 0 and 1. That is,\n\nFibonacci(0) = 0,\nFibonacci(1) = 1,\nFibonacci(n) = Fibonacci(n - 1) + Fibonacci(n - 2) for n > 1.\n\nHere is a simple implementation of the function in JavaScript:\n\n```javascript\nfunction fibonacci(n) {\n    if (n <= 1) {\n        return n;\n    } else {\n        return fibonacci(n - 1) + fibonacci(n - 2);\n    }\n}\n```\n\nThis function uses recursion to calculate the nth Fibonacci number. However, this","logprobs":null,"finish_reason":"length"}],

Prompt 42==> Problem: Write a function in Ruby to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime. 
 {"id":"cmpl-ebf17746ad324470928b38655a4a5d6e","object":"text_completion","created":520085,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef is_prime(num)\n  if num <= 1\n    return false\n  elsif num <= 3\n    return true\n  elsif num % 2 == 0 || num % 3 == 0\n    return false\n  end\n\n  i = 5\n  while i * i <= num\n    if num % i == 0 || num % (i + 2) == 0\n      return false\n    end\n    i += 6\n  end\n\n  return true\nend\n\nputs is_prime(1)  # false\nputs is_prime(2)  # true\nputs is_prime(3)  # true\nputs is_prime(4)  # false\nputs is_prime(5)  # true\nputs is_prime(10) # false\nputs is_prime(11) # true\nputs is_prime","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":38,"total_tokens":238,"com

Prompt 62==> Problem: Build a function in CSS to toggle between light and dark modes for a website.Input: None.Output: None (update the website theme). 
 {"id":"cmpl-925f4d18d1ee40748a153141f0b97bbd","object":"text_completion","created":520088,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple CSS function to toggle between light and dark modes:\n\n```css\n@media (prefers-color-scheme: dark) {\n  body {\n    background-color: #333;\n    color: #fff;\n  }\n}\n\n@media (prefers-color-scheme: light) {\n  body {\n    background-color: #fff;\n    color: #333;\n  }\n}\n```\n\nThis CSS function uses the `prefers-color-scheme` media feature to detect the user's preferred color scheme. If the user's preferred color scheme is dark, the body background color will be set to a dark grey (#333) and the text color will be set to white (#fff). If the user's preferred color scheme is light, t

Prompt 69==> Problem: Create a function in CSS to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element). 
 {"id":"cmpl-b9f8d279f2804460892033e20d970103","object":"text_completion","created":520094,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple CSS function that toggles a class on an HTML element when a button is clicked:\n\n```css\n@keyframes toggleClass {\n  0% {\n    opacity: 0;\n  }\n  100% {\n    opacity: 1;\n  }\n}\n\nbutton {\n  animation: toggleClass 2s;\n}\n\nbutton:hover {\n  background-color: #f0f0f0;\n}\n\n.myClass {\n  color: red;\n}\n\nbutton:active + .myClass {\n  color: blue;\n}\n```\n\nIn this example, when the button is clicked, the class \"myClass\" is toggled on the HTML element. The animation makes the button fade in over 2 seconds. The hover effe

Prompt 76==> Problem: Build a function in react to populate an HTML table with data from an array of objects.Input: Array of objects, each representing a row in the table.Output: None (update the HTML table). 
 {"id":"cmpl-31b09341666f49f190138a9ad0b2dd0d","object":"text_completion","created":520101,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can do this in React:\n\n```jsx\nimport React from 'react';\n\nclass Table extends React.Component {\n  constructor(props) {\n    super(props);\n    this.state = {\n      data: [\n        {id: 1, name: 'John', age: 25},\n        {id: 2, name: 'Jane', age: 30},\n        {id: 3, name: 'Bob', age: 35}\n      ]\n    };\n  }\n\n  render() {\n    return (\n      <table>\n        <thead>\n          <tr>\n            <th>ID</th>\n            <th>Name</th>\n            <th>Age</th>\n          </tr>\n        </thead>\n   

Prompt 89==> Problem: Create a function in angular to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element). 
 {"id":"cmpl-18ff364367524fa8a65b145a596e8067","object":"text_completion","created":520104,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nExample:\n\nHTML:\n\n```html\n<button (click)=\"toggleClass('myElement', 'myClass')\">Toggle Class</button>\n<div id=\"myElement\">Hello World</div>\n```\n\nAngular Component:\n\n```typescript\nimport { Component } from '@angular/core';\n\n@Component({\n  selector: 'app-root',\n  templateUrl: './app.component.html',\n  styleUrls: ['./app.component.css']\n})\nexport class AppComponent {\n  toggleClass(elementId: string, className: string) {\n    const element = document.getElementById(elementId);\n    if (element) {\n      element.classList.to

Prompt 97==> Problem: Write a function in jQuery that dynamically creates an HTML element and appends it to the document.Input: String, element type (e.g., 'div', 'button').Output: None (update the HTML document). 
 {"id":"cmpl-d5cb6edd9e464a908a528f384b2cfab1","object":"text_completion","created":520109,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple jQuery function that creates an HTML element and appends it to the document:\n\n```javascript\nfunction createElement(elementType, text) {\n    var element = $('<' + elementType + '>');\n    element.text(text);\n    $('body').append(element);\n}\n```\n\nYou can call this function like this:\n\n```javascript\ncreateElement('div', 'Hello, world!');\n```\n\nThis will create a new div element with the text \"Hello, world!\" and append it to the body of the document.","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_token

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    index=0
    global df
    for i in range(1):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
            futures = [executor.submit(inference, srno, payload) for srno,prompt in enumerate(prompts, start=index)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)